In [3]:
import pandas as pd

data = pd.read_csv("C:/datasets/Ae_data.csv")
data.head()

,time,y,x1,x2,x3,x4,x5,x6,x7,x8,...,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61
0,5/1/99 0:00,0,0.376665,-4.596435,-4.095756,13.497687,-0.118830,-20.669883,0.000732,-0.061114,...,10.091721,0.053279,-4.936434,-24.590146,18.515436,3.473400,0.033444,0.953219,0.006076,0
1,5/1/99 0:02,0,0.475720,-4.542502,-4.018359,16.230659,-0.128733,-18.758079,0.000732,-0.061114,...,10.095871,0.062801,-4.937179,-32.413266,22.760065,2.682933,0.033536,1.090502,0.006083,0
2,5/1/99 0:04,0,0.363848,-4.681394,-4.353147,14.127997,-0.138636,-17.836632,0.010803,-0.061114,...,10.100265,0.072322,-4.937924,-34.183774,27.004663,3.537487,0.033629,1.840540,0.006090,0
3,5/1/99 0:06,0,0.301590,-4.758934,-4.023612,13.161566,-0.148142,-18.517601,0.002075,-0.061114,...,10.104660,0.081600,-4.938669,-35.954281,21.672449,3.986095,0.033721,2.554880,0.006097,0
4,5/1/99 0:08,0,0.265578,-4.749928,-4.333150,15.267340,-0.155314,-17.505913,0.000732,-0.061114,...,10.109054,0.091121,-4.939414,-37.724789,21.907251,3.601573,0.033777,1.410494,0.006105,0


1. 데이터 전처리:

lyingRigh과 lyingBack을 정상 데이터로 사용하고, jumping을 비정상 데이터로 추가합니다.

lying 데이터만으로 AE를 훈련합니다.

In [4]:
# 확인할 활동 종류와 데이터 분포
activity_counts = data['y'].value_counts()

# 활동 종류와 각 개수 출력
activity_counts

y
0    18274
1      124
Name: count, dtype: int64

lyingRigh과 lyingBack은 y = 0에서 선택된 데이터로 가정하고 정상 데이터로 사용합니다.

y = 1은 jumping 데이터로 가정하여 비정상 데이터로 사용합니다.

In [5]:
# 정상 데이터 (lying)와 비정상 데이터 (jumping) 분리
lying_data = data[data['y'] == 0]
jumping_data = data[data['y'] == 1]

# lying 데이터를 AE 훈련용으로 준비
X_train = lying_data.drop(columns=['time', 'y'])

# 테스트 데이터: lying + jumping 혼합
test_data = pd.concat([lying_data, jumping_data])
X_test = test_data.drop(columns=['time', 'y'])
y_test = test_data['y']

# 데이터 분리 결과 확인
(X_train.shape, X_test.shape, y_test.value_counts())

((18274, 61),
 (18398, 61),
 y
 0    18274
 1      124
 Name: count, dtype: int64)

In [9]:
lying_sample = lying_data.sample(n=960, random_state=42)  # 총 960개 (480 + 480)
X_train_sampled = lying_sample.drop(columns=['time', 'y'])

# 샘플링된 데이터 크기 확인
X_train_sampled.shape

(960, 61)

In [10]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

# TensorFlow 기반 Keras로 오토인코더 구현
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

import numpy as np

scaler = MinMaxScaler()
# 훈련 데이터 (lying 데이터만)
X_train_scaled_sampled = scaler.fit_transform(X_train_sampled)

# 테스트 데이터
X_test_scaled = scaler.transform(X_test)

# 오토인코더 모델 설계
input_dim = X_train_scaled_sampled.shape[1]
input_layer = Input(shape=(input_dim,))
encoder = Dense(32, activation="relu")(input_layer)
encoder = Dense(16, activation="relu")(encoder)
decoder = Dense(32, activation="relu")(encoder)
decoder = Dense(input_dim, activation="sigmoid")(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

# 모델 컴파일 및 학습
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
history = autoencoder.fit(
    X_train_scaled_sampled, X_train_scaled_sampled,
    epochs=50, batch_size=32, shuffle=True, validation_split=0.1, verbose=1
)

# 테스트 데이터 재구성
reconstructed = autoencoder.predict(X_test_scaled)
reconstruction_error = np.mean((X_test_scaled - reconstructed) ** 2, axis=1)

# 임계값 설정 및 분류
threshold = np.mean(reconstruction_error) + 2 * np.std(reconstruction_error)
y_pred = (reconstruction_error > threshold).astype(int)

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
accuracy

Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0872 - val_loss: 0.0653
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0542 - val_loss: 0.0367
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0330 - val_loss: 0.0304
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0274 - val_loss: 0.0269
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0241 - val_loss: 0.0247
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0231 - val_loss: 0.0234
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0220 - val_loss: 0.0225
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0211 - val_loss: 0.0216
Epoch 9/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0196 - val_loss: 0.0203
Epoch 10/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0182 - val_loss: 0.0189
Epoch 11/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0166 - val_loss: 0.0176
Epoch 12/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0167 - val_lo

0.9936949668442222